# <font color='#2B17B0'>CIENCIA DE DATOS APLICADA A FINANZAS</font>
Sesión 7, 2023-10-23/24<br>
Carlos Crespo Elizondo<br>
EGADE BUSINESS SCHOOL

# <font color='#2B17B0'>Librerías a utilizar en clases</font>

In [ ]:
from matplotlib.patches import FancyArrow
from matplotlib.patches import Circle
import matplotlib.pyplot as plt
import herramientas_financieras as hf
import pandas as pd
import numpy as np
import itertools
import seaborn as sns


# sns.set_style('darkgrid')

In [ ]:
%load_ext autoreload
%autoreload 2

# <font color='#2B17B0'>Previo a empezar la clase:</font>

1. _Leer los componentes del IPC directamente del archivo de Excel de la página de BMV/S&P IPC (desarrollarlo en otra libreta)_
    * _Link: https://www.spglobal.com/spdji/en/indices/equity/sp-bmv-ipc/#overview_
1. _Mejorar la función precios para que pueda leer los precios de una o varias emisoras_
    1. Incluir:
        1. docstring
        1. parámetros preestablecidos (accion=IPC, fecha2= la fecha de hoy y fecha1= hoy hace un año
    * **_NOTA: Actualmente la función puede leer solo una emisora_**

# <font color='#2B17B0'>Portafolios de Inversión</font>

## <font color='#2B17B0'>Retornos y volatilidad de 2 activos</font>

In [ ]:
df_2activos = pd.DataFrame({
    "rojo":{'ret':.04, 'sigma':.2},
    "azul":{'ret':.12, 'sigma':.3}})
df_2activos

In [ ]:
hf.fig_2activos(df_2activos)

### Portafolio compuesto por 50% <font color='blue'>activo azul</font> y 50% <font color='red'>activo rojo</font>

In [ ]:
hf.fig_pregunta(df_2activos)

### Cálculo del retorno de un portafolio de 2 activos

$$\Large R_{w_A, w_B}=w_A*R_A + w_B*R_B$$

### Cálculo de la volatilidad de un portafolio de 2 activos (fórmula teórica)

$$\Large \sigma^2_{w_A, w_B}=\sigma^2_{A}w^2_{A}+\sigma^2_{B}w^2_{B}+2w_Aw_B\sigma_A\sigma_B\rho_{A,B}$$

$$\Large \sigma_{w_a, w_b}=\sqrt{\sigma^2_{w_A, w_B}}$$

In [ ]:
hf.fig_portafolio(df_2activos, .1)

In [ ]:
hf.fig_port_dif_correlacion(df_2activos)

## <font color='#2B17B0'>Construir portafolios utilizando Simulación MonteCarlo</font>

Para el desarrollo del primer portafolio de inversión, vamos a seguir los siguiente pasos:

1. Importar librerías
1. Importar precios de cierre
1. Calcular retornos diarios y anuales
1. Calcular $\sigma$ diaria y anual
1. Graficar dispersiones de los retornos anuales Vs $\sigma$ anual
1. Seleccionar de manera cuantitativa las acciones que formaran el portafolio (Sharp Ratio)
1. Calcular matriz de covarianza  de las acciones seleccionadas
1. Crear array NumPy para guardar los resultados de las simulaciones.
1. Realizar simulación Monte Carlo
1. Visualizar portafolio
1. Visualizar resultados óptimos

### <font color='royalblue'>2. Importar precios de cierre</font>

### <font color='royalblue'>3. Calcular retornos diarios y anuales</font>

#### Retornos diarios

#### Retornos anualizados

### <font color='royalblue'>4. Calcular $\sigma$ diaria y anual</font>

#### Sigma diaria

#### Sigma anual

### <font color='royalblue'>5. Graficar dispersiones de los retornos anuales Vs $\sigma$ anual</font>

### <font color='royalblue'>6. Seleccionar de manera cuantitativa las acciones que formaran el portafolio</font>

#### Sharpe Ratio
El Sharp Ratio, mide el rendimiento adicional que genera un activo por encima de la tasa de retorno libre de riesgo, ajustado por su volatilidad.<br><br>
$$SharpRatio = \frac{r_x - r_f}{\sigma_x}$$<br>
Como tasa libre de riesgo utilizaremos CETES a 28 dias $\approx11\%$ anual.

### <font color='royalblue'>7. Calcular matriz de covarianza  de las acciones seleccionadas</font>

### <font color='royalblue'>8. Crear array NumPy para guardar los resultados de las simulaciones</font>
Incluir las siguientes columnas:
1. \# de simulación
  1. Retorno generado
  1. $\sigma$ del portafolio
  1. Sharp Ratio del portafolio
  1. y el peso (w) de cada acción que forman el portafolio

### <font color='royalblue'>9. Realizar simulación Monte Carlo</font>

### <font color='gray'>Cálculo $E(R_p)$, $\sigma^2$ de un portafolio</font>
Para calcular el retorno de un portafolio de *"n"* activos:<br>
$E(R_p) = \displaystyle\sum_{i=1}^{n}{w_iE(R_i)}\implies Rw$<br><br>
Para calcular la $\sigma^2$ de un portafolio de dos activos -Recordar que la nombre como fórmula teórica-:<br>
$\sigma_p^2 = w_1^2\sigma_1^2 + w_2^2\sigma_2^2 + 2w_1w_2\sigma_1\sigma_2\rho_{12}$<br><br>
Recordememos que la fórmula de la correlación es:<br><br>
$\large\rho=\frac{Cov_{i,j}}{\sigma_i \sigma_j}$<br><br>
Podemos simplificar la fórmula de la varianza eliminando sustituyendo la fórmula de la $\rho$ y simplificándola a:<br><br>
$\sigma_p^2 = w_1^2\sigma_1^2 + w_2^2\sigma_2^2 + 2w_1w_2\sigma_1\sigma_2\frac{Cov_{1,2}}{\sigma_1 \sigma_2}\implies \sigma_p^2 = w_1^2\sigma_1^2 + w_2^2\sigma_2^2 + 2w_1w_2\sigma_{1,2}$<br><br>
Para calcular la $\sigma^2$ de un portafolio de *"n"* activos:<br>
$$
\begin{array}{rcl}
\sigma_p^2 & = & \displaystyle\sum_{i=1}^{n} {w_i^2\sigma_i^2} +  \displaystyle\sum_{i=1}^{n} \displaystyle\sum_{j\neq i}^{n}w_iw_j\sigma_i\sigma_j\rho_{ij}\\
& = & \displaystyle\sum_{i=1}^{n} \displaystyle\sum_{j=i}^{n}w_iw_j\sigma_{i,j}\\
& = & w^T\sum w 
\end{array}
$$
Donde $\sum$ es la matriz de varianza - covarianza

### <font color='gray'>Renombrar columnas</font>

### <font color='gray'>Identificar portafolio con el máximo Sharp Ratio y el de mínima varianza </font>

### <font color='royalblue'>10. Visualizar portafolio</font>